# Fichiers de départ

Nous proposons de travailler sur des données décrivant des films.

L'un des fichiers de départ, "movies.csv" est disponible sur:
https://grouplens.org/datasets/movielens/
au format CSV. 
Il contient des id de film, le titre et les genres associés.

Le deuxième fichier, "film_v2.pkl", est récupéré depuis le site TMdb. Malheuresement, le lien du fichier est désormais inutilisable: http://webia.lip6.fr/~guigue/act_v2.pkl.
On se concentrera sur le booléen adulte si le film est interdit aux mineurs.

Des soucis de format sont a prendre en compte tel le titre dans le fichier movies.csv, par exemple plusieurs titres ne respectent pas le format "titre" + "1 espace" + "année de sortie en parenthèse".
Il y aussi des doublons pour une paire titre/année. Il est donc nécessaire nettoyer les données avant de faire une jointure sur ces fichiers.


# Transformation de movies.csv

In [1]:
import re
import numpy as np
import pandas as pd
import pickle as pkl

In [2]:
movies = pd.read_csv('data/movies.csv', encoding='utf8')
#supprime les lignes avec des valeurs à None
movies = movies.dropna()

## Doublons de pair titre/année
Pour un même film et une même année, il y a des doublons avec des genres différents.
On garde alors une ligne avec le nombre de genre le plus elevé.

In [3]:
movies[movies.title == 'Emma (1996)']

,movieId,title,genres
823,838,Emma (1996),Comedy|Drama|Romance
9135,26958,Emma (1996),Romance


In [4]:
#il y a 32 doublons dans le fichier
movies[movies.duplicated(['title'], keep=False)].count()

movieId    32
title      32
genres     32
dtype: int64

In [5]:
def sort_df(df, column_idx, key):
    '''Prend un daataframe, une colonne et une fonction de trie ,
    retourne un dataframe triée par la colonne donnée selon la fonction de trie'''

    col = df[column_idx]
    df = df.iloc[[i[1] for i in sorted(zip(col,range(len(col))), key=key)]]
    return df

In [6]:
#transforme le string genre en tableau de genre
movies.genres=movies.genres.str.split('|')
#on trie par rapport a la taille du tableau
cmp = lambda x: len(x)  
#trie par la taille du tableu genre et garde le plus long
movies= sort_df(movies,'genres',cmp).drop_duplicates('title', keep='first')  
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


## Format du titre
Plusieurs titres ont des formats ambiguës, il faut les supprimmer.

In [7]:
movies[movies.title.str.contains("Skokie")].iloc[0]['title']

'Skokie\xa0(1981)'

In [8]:
movies[movies.title.str.contains(r".* \(\d+\)")== False][['title']].head()

,title
7760,Skokie (1981)
9071,Deadly Advice(1994)
10593,Babylon 5
15646,"Millions Game, The (Das Millionenspiel)"
17341,"Bicycle, Spoon, Apple (Bicicleta, cullera, poma)"


In [9]:
#garde les titres avec le format "titre" + "1 espace" + "année de sortie en parenthèse"
movies= movies[movies.title.str.contains(r".* \(\d+\)")]

## Separation du titre et de l'année

In [10]:
def f(x):
    '''Prend une ligne de Dataframe en parametre et sépare le titre et la data et 2 colonnes. '''   
    p= re.search(r"(.*) \((\d+)\)", x)
    return pd.Series([p.group(1),p.group(2)])

In [11]:
movies[['title','year']] = movies.title.apply(f)
movies.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [12]:
#Il n'y a plus de doublons pour une paire titre/année
movies[movies.duplicated(['title','year'], keep=False)]

,movieId,title,genres,year


# Jointure entre les deux fichiers
Il y a deux attributs "title" et "original_title" dans film_v2.pkl, on regarde le nombre de film restant après la jointure avec movies.csv.

In [13]:
films = pkl.load(open("data/film_v2.pkl", "rb"))
films = pd.DataFrame(films)
films = films.dropna()
films2 = films.copy()

films= films[['adult','title','release_date']]
films= films.rename(columns = {'release_date':'year'}) #on renomme pour la jointure
#on retire le mois et la date car elle n'est pas présente dans movies.csv
films.year = films.year.str.replace(r"-.*","") 
#on retire les doublons titre/année
films = films.drop_duplicates(['title','year']) 

films2= films2[['original_title','release_date']]
films2= films2.rename(columns = {'release_date':'year','original_title':'title'}) 
films2.year = films2.year.str.replace(r"-.*","")  
films2 = films2.drop_duplicates(['title','year'])

#jointure avec movies.csv
mf= pd.merge(movies,films, on= ['title','year'],how='inner')
mf2= pd.merge(movies,films2, on= ['title','year'],how='inner')

On utilise l'attribut 'title' car il plus de film correspondant après la jointure avec cet attribut

In [14]:
print(mf.shape[0], mf2.shape[0])

13796 13175


Le tableau est "propre", il n'y a plus de doublons.

In [15]:
mf[mf.duplicated(['title','year'], keep=False)]

,movieId,title,genres,year,adult


In [16]:
mf.head()

,movieId,title,genres,year,adult
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,False
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,False
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,False
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,False
4,5,Father of the Bride Part II,[Comedy],1995,False


Pour l'association Appartient, on crée les paires movieId/genre

In [17]:
mo= mf[['movieId','genres']].copy()
mo=mo.explode('genres') 
#mo.genres= pd.Categorical(mo.genres)
#mo['code'] = mo.genres.cat.codes

Données pour la table FILMS

In [18]:
mf.drop('genres', axis=1).head()

,movieId,title,year,adult
0,1,Toy Story,1995,False
1,2,Jumanji,1995,False
2,3,Grumpier Old Men,1995,False
3,4,Waiting to Exhale,1995,False
4,5,Father of the Bride Part II,1995,False


Données pour la table APPARTIENT

In [19]:
mo.head()

,movieId,genres
0,1,Adventure
0,1,Animation
0,1,Children
0,1,Comedy
0,1,Fantasy


Données pour la table GENRES

In [20]:
mo.drop('movieId', axis=1).drop_duplicates().head()

,genres
0,Adventure
0,Animation
0,Children
0,Comedy
0,Fantasy


Enfin, la création des fichiers.

In [21]:
mf.columns = ['id_film', 'titre', 'genres', 'annee', 'adulte']
mo.columns = ['movieId', 'genre']

In [22]:
mf.head()

,id_film,titre,genres,annee,adulte
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,False
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,False
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,False
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,False
4,5,Father of the Bride Part II,[Comedy],1995,False


In [23]:
mo.head()

,movieId,genre
0,1,Adventure
0,1,Animation
0,1,Children
0,1,Comedy
0,1,Fantasy


In [24]:
mf['moy_note'] = 0
mf['nb_note'] = 0

In [26]:
mf.drop('genres', axis=1).to_csv(r'movies14k.csv', index=False, header=True)
mo.to_csv(r'appartient.csv', index=False, header=True)
mo.drop('movieId', axis=1).drop_duplicates().to_csv(r'genres.csv', index=False, header=True)